This file useses the purchases of the four FAs that we could find (2239-5-LR21, 2239-8-LR23, 2239-4-LR17 and ID 2239-20-LP13 ) and does the following things: 
1. Joins them 
2. merges them with the car data
3. estimates a demand model

In [234]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import seaborn as sns
from pandasgui import show
import socket
from unidecode import unidecode
import unicodedata

### Functions

In [255]:
def clean_text(text):
    if isinstance(text, str):
        return unidecode(text)
    return text


def fix_encoding(text):
    if isinstance(text, str):
        replacements = {
            'Ã³': 'ó',
            'Ã­': 'í',
            'Ã¡': 'á',
            'Ã©': 'é',
            'Ã±': 'ñ',
            'Ãº': 'ú',
            'Ã¼': 'ü',
            'Ã': 'í'
        }
        for wrong, right in replacements.items():
            text = text.replace(wrong, right)
    return text

def process_prices(df):
    """
    Process prices in a dataframe according to specific rules using vectorized operations:
    1. Process all prices in scientific notation first
    2. For remaining prices with commas, remove everything after the comma
    3. For USD prices, multiply by 700
    
    Parameters:
    df (pd.DataFrame): Input dataframe with Moneda and Precio Unitario columns
    
    Returns:
    pd.DataFrame: Processed dataframe
    pd.DataFrame: Rows with processing issues
    """
    # Create a copy to avoid modifying the original
    df_processed = df.copy()
    
    # Ensure the required columns exist
    required_cols = ['Moneda', 'Precio Unitario']
    if not all(col in df_processed.columns for col in required_cols):
        raise ValueError(f"Missing required columns. Need: {required_cols}")
    
    # Convert Precio Unitario to string type for string operations
    df_processed['Precio Unitario'] = df_processed['Precio Unitario'].astype(str)
    
    # Create masks for different conditions
    has_scientific_notation = df_processed['Precio Unitario'].str.contains('e', case=False, na=False)
    has_comma_mask = df_processed['Precio Unitario'].str.contains(',', na=False)
    is_usd_mask = df_processed['Moneda'] == 'USD'
    
    # 1. Process scientific notation first
    scientific_values = df_processed[has_scientific_notation]['Precio Unitario']
    if not scientific_values.empty:
        # Replace comma with dot for proper float conversion
        scientific_values = scientific_values.str.replace(',', '.')
        df_processed.loc[has_scientific_notation, 'Precio Unitario'] = (
            pd.to_numeric(scientific_values, errors='coerce')
        )
    
    # 2. Process remaining prices with commas (excluding already processed scientific notation)
    remaining_comma_mask = has_comma_mask & ~has_scientific_notation
    if any(remaining_comma_mask):
        df_processed.loc[remaining_comma_mask, 'Precio Unitario'] = (
            df_processed.loc[remaining_comma_mask, 'Precio Unitario']
            .str.split(',')
            .str[0]
        )
    
    # Convert all prices to numeric
    df_processed['Precio Unitario'] = pd.to_numeric(df_processed['Precio Unitario'], errors='coerce')
    
    # 3. Multiply USD prices by 700
    df_processed.loc[is_usd_mask, 'Precio Unitario'] = df_processed.loc[is_usd_mask, 'Precio Unitario'] * 700
    
    # Find problematic rows (where conversion to numeric failed)
    problematic_rows = df_processed[df_processed['Precio Unitario'].isna()].copy()
    print(f"Number of problematic rows: {len(problematic_rows)}")

    return df_processed


### Read the purchases for the FAs and do some data cleaning

In 2017 we have also leases and repairs of cars. This has to be fixed. 

In [256]:
transac_cm_2013_path = os.path.join('..', 'interm_data', 'yearly_data', 'Transacciones', 'transacciones_cm_2013.csv')
transac_cm_2013_df = pd.read_csv(transac_cm_2013_path)
print(f"Dimensions of transac_cm_2013_df: {transac_cm_2013_df.shape}")
transac_cm_2013_df.head()

Dimensions of transac_cm_2013_df: (0, 1)


,Unnamed: 0


In [257]:
transac_cm_2017_path = os.path.join('..', 'interm_data', 'yearly_data', 'Transacciones', 'transacciones_cm_2017.csv')
transac_cm_2017_df = pd.read_csv(transac_cm_2017_path)
transac_cm_2017_df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'], inplace=True)
print(f"Dimensions of transac_cm_2017_df: {transac_cm_2017_df.shape}")

# Extract the year from the 'Fecha Envío OC' column
transac_cm_2017_df['year'] = pd.to_datetime(transac_cm_2017_df['Fecha Envío OC']).dt.year

# Count the number of observations for each year
year_counts = transac_cm_2017_df['year'].value_counts().sort_index()
print(year_counts)

nro_counts = transac_cm_2017_df['Nro Licitación Pública'].value_counts().sort_index() 
print(nro_counts)

transac_cm_2017_df.head()
#show(transac_cm_2017_df)

transac_cm_2017_df = process_prices(transac_cm_2017_df)

Dimensions of transac_cm_2017_df: (19997, 45)
year
2018    3574
2019    6056
2020    5174
2021    5193
Name: count, dtype: int64
Nro Licitación Pública
2239-4-LR17    19997
Name: count, dtype: int64
Number of problematic rows: 0


In [258]:
transac_cm_2021_path = os.path.join('..', 'interm_data', 'yearly_data', 'Transacciones', 'transacciones_cm_2021.csv')
transac_cm_2021_df = pd.read_csv(transac_cm_2021_path)
transac_cm_2021_df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'], inplace=True)
print(f"Dimensions of transac_cm_2021_df: {transac_cm_2021_df.shape}")

 # Fix weird characters in columns and in text 
#transac_cm_2021_df.columns = [clean_text(col) for col in transac_cm_2021_df.columns]
#for column in transac_cm_2021_df.select_dtypes(include=['object']).columns:
#    transac_cm_2021_df[column] = transac_cm_2021_df[column].apply(clean_text)



transac_cm_2021_df.columns = [fix_encoding(col) for col in transac_cm_2021_df.columns]
for column in transac_cm_2021_df.select_dtypes(include=['object']).columns:
    transac_cm_2021_df[column] = transac_cm_2021_df[column].apply(fix_encoding)


transac_cm_2021_df['year'] = pd.to_datetime(transac_cm_2021_df['Fecha Envío OC']).dt.year


# Count the number of observations for each year
year_counts = transac_cm_2021_df['year'].value_counts().sort_index()
print(year_counts)

nro_counts = transac_cm_2021_df['Nro Licitación Pública'].value_counts().sort_index() 

print(nro_counts)

transac_cm_2021_df = process_prices(transac_cm_2021_df)
transac_cm_2021_df.head()



Dimensions of transac_cm_2021_df: (1071, 45)
year
2021      7
2022    277
2023    721
2024     66
Name: count, dtype: int64
Nro Licitación Pública
2239-5-LR21    1071
Name: count, dtype: int64
Number of problematic rows: 0


,Nro Licitación Pública,Id Convenio Marco,Convenio Marco,CodigoOC,NombreOC,Fecha Envío OC,EstadoOC,Proviene de Gran Compra,idGranCompra,Especificación del Comprador,...,Nombre Empresa,Comuna del Proveedor,Región del Proveedor,Observaciones,Forma de Pago,Orgcode_Comprador,Entcode_Comprador,Orgcode_Proveedor,Entcode_Proveedor,year
0,2239-5-LR21,5800296.0,CM Adquisición de Vehículos SUV y Camionetas (...,2564-270-CM21,Orden de Compra: 2564-270-CM21,2021-10-20,Enviada a Proveedor,No,NaN,(1840093) CAMIONETAS MAXUS T60 T60 DX 4X4 MAC...,...,Mediterraneo Automotores S.A.,NaN,NaN,Orden de Compra codigo: 2564-270-CM21 dirigida...,30 días contra la recepción conforme de la fac...,3555.0,92414.0,824156,27979,2021
1,2239-5-LR21,5800296.0,CM Adquisición de Vehículos SUV y Camionetas (...,4360-75-CM21,Camioneta maxus d/c 4x4,2021-12-24,Enviada a Proveedor,No,NaN,(1840095) CAMIONETAS MAXUS T60 T60 DX 4X4 MAC...,...,Mediterraneo Automotores S.A.,NaN,NaN,Orden de Compra codigo: 4360-75-CM21 dirigida ...,30 días contra la recepción conforme de la fac...,5277.0,135561.0,824156,27979,2021
2,2239-5-LR21,5800296.0,CM Adquisición de Vehículos SUV y Camionetas (...,3140-819-CM21,"Orden de Compra: 3140-819-CM21, ADQUISICIíN D...",2021-12-21,Enviada a Proveedor,No,NaN,(1848052) CAMIONETA NISSAN NAVARA DC SE 2.3D M...,...,SALINAS Y FABRES SOCIEDAD ANONIMA,NaN,NaN,Orden de Compra codigo: 3140-819-CM21 dirigida...,30 días contra la recepción conforme de la fac...,4061.0,113735.0,26569,26778,2021
3,2239-5-LR21,5800296.0,CM Adquisición de Vehículos SUV y Camionetas (...,1084170-110-CM21,Camioneta para Uso del SLEP,2021-12-28,Recepción Conforme,No,NaN,(1840060) CAMIONETAS MAHINDRA PIK UP PICK UP R...,...,BRUNO FRITSCH S.A.,NaN,NaN,1.\tAceptar la OC y facturar después de entreg...,30 días contra la recepción conforme de la fac...,1084170.0,1675231.0,24507,24823,2021
4,2239-5-LR21,5800296.0,CM Adquisición de Vehículos SUV y Camionetas (...,4236-722-CM21,Orden de Compra: 4236-722-CM21,2021-12-29,Aceptada,No,NaN,(1848008) CAMIONETA TOYOTA HILUX 4X4 2.4 MT DX...,...,Maritano y Ebensperger Ltda.,NaN,NaN,Orden de Compra codigo: 4236-722-CM21 dirigida...,30 días contra la recepción conforme de la fac...,5154.0,127036.0,62896,61164,2021


In [259]:
transac_cm_2023_path = os.path.join('..', 'interm_data',  'yearly_data', 'Transacciones', 'transacciones_cm_2023.csv')
transac_cm_2023_df = pd.read_csv(transac_cm_2023_path)
transac_cm_2023_df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'], inplace=True)
print(f"Dimensions of transac_cm_2023_df: {transac_cm_2023_df.shape}")

transac_cm_2023_df.columns = [fix_encoding(col) for col in transac_cm_2023_df.columns]
for column in transac_cm_2023_df.select_dtypes(include=['object']).columns:
    transac_cm_2023_df[column] = transac_cm_2023_df[column].apply(fix_encoding)


transac_cm_2023_df['year'] = pd.to_datetime(transac_cm_2023_df['Fecha Envío OC']).dt.year


# Count the number of observations for each year
year_counts = transac_cm_2023_df['year'].value_counts().sort_index()
print(year_counts)

nro_counts = transac_cm_2023_df['Nro Licitación Pública'].value_counts().sort_index() 
print(nro_counts)

transac_cm_2023_df = process_prices(transac_cm_2023_df)


transac_cm_2023_df.head()

Dimensions of transac_cm_2023_df: (366, 45)
year
2024    366
Name: count, dtype: int64
Nro Licitación Pública
2239-8-LR23    366
Name: count, dtype: int64
Number of problematic rows: 0


,Nro Licitación Pública,Id Convenio Marco,Convenio Marco,CodigoOC,NombreOC,Fecha Envío OC,EstadoOC,Proviene de Gran Compra,idGranCompra,Especificación del Comprador,...,Nombre Empresa,Comuna del Proveedor,Región del Proveedor,Observaciones,Forma de Pago,Orgcode_Comprador,Entcode_Comprador,Orgcode_Proveedor,Entcode_Proveedor,year
0,2239-8-LR23,5802342.0,Convenio Marco de Adquisición de Vehículos,4062-187-CM24,Orden de Compra: 4062-187-CM24,2024-03-12,Recepcion Conforme,No,NaN,(2042525) CAMIONETA MAXUS T60 E6 GL 4X4 MT 202...,...,AUTOMOTRIZ SERVIMAQ SPA,NaN,NaN,Orden de Compra codigo: 4062-187-CM24 dirigida...,30 dias contra la recepcion conforme de la fac...,4982,120001,100329,97856,2024
1,2239-8-LR23,5802342.0,Convenio Marco de Adquisición de Vehículos,3577-41-CM24,Orden de Compra: 3577-41-CM24,2024-03-14,Aceptada,No,NaN,(2041399) CAMIONETA TOYOTA HILUX DX 2.4 4X4 MT...,...,BRUNO FRITSCH S.A.,NaN,NaN,Orden de Compra codigo: 3577-41-CM24 dirigida ...,30 dias contra la recepcion conforme de la fac...,4497,115285,24507,24823,2024
2,2239-8-LR23,5802342.0,Convenio Marco de Adquisición de Vehículos,3030-177-CM24,CAMIONETA JMC PARA DEPTO. DE SALUD,2024-03-14,Aceptada,No,NaN,(2041534) CAMIONETA JMC VIGUS NEW WORK 4X4 MT ...,...,AUTOMOTRIZ CORDILLERA S.A.,NaN,NaN,CAMIONETA JMC VIGUS NEW WORK 4X4 MT 2024 PARA ...,30 dias contra la recepcion conforme de la fac...,3978,113252,32639,32414,2024
3,2239-8-LR23,5802342.0,Convenio Marco de Adquisición de Vehículos,2381-308-CM24,ADQUISICIíN DE 01 CAMIONETA TODO TERRENO PARA...,2024-03-18,Aceptada,No,NaN,(2041392) CAMIONETA TOYOTA HILUX DX 2.4 4X4 MT...,...,BRUNO FRITSCH S.A.,NaN,NaN,ADQUISICIíN DE 01 CAMIONETA TODO TERRENO PARA...,30 dias contra la recepcion conforme de la fac...,3373,86872,24507,24823,2024
4,2239-8-LR23,5802342.0,Convenio Marco de Adquisición de Vehículos,316-212-CM24,Orden de Compra: 316-212-CM24,2024-03-20,Aceptada,No,NaN,(2041566) CAMIONETA JMC VIGUS NEW PLUS 4X4 MT ...,...,AUTOMOTRIZ CORDILLERA S.A.,NaN,NaN,Orden de Compra codigo: 316-212-CM24 dirigida ...,30 dias contra la recepcion conforme de la fac...,1313,7233,32639,32414,2024


### Join the purchases from all the different FAs

In [260]:
# Concatenate the dataframes vertically
transac = pd.concat([transac_cm_2017_df, transac_cm_2021_df, transac_cm_2023_df], axis=0)

# Reset the index if needed
transac_df = transac.reset_index(drop=True)
print(transac_df.columns)
print(f"Dimensions of transac_df: {transac_df.shape}")
transac_df.head(2)
#show(transac_df)

Index(['Nro Licitación Pública', 'Id Convenio Marco', 'Convenio Marco',
       'CodigoOC', 'NombreOC', 'Fecha Envío OC', 'EstadoOC',
       'Proviene de Gran Compra', 'idGranCompra',
       'Especificación del Comprador', 'IDProductoCM', 'Producto',
       'Nombre Producto ONU', 'Tipo de Producto', 'Marca', 'Modelo',
       'Precio Unitario', 'Cantidad', 'TotaLínea(Neto)', 'Moneda',
       'Monto Total OC Neto', 'Descuento Global OC', 'Cargos Adicionales OC',
       'Subtotal OC', 'Impuestos', 'Monto Total OC', 'Rut Unidad de Compra',
       'Unidad de Compra', 'Razón Social Comprador', 'Dirección Unidad Compra',
       'Comuna Unidad Compra', 'Región Unidad de Compra', 'Institución',
       'Sector', 'Rut Proveedor', 'Nombre Proveedor Sucursal',
       'Nombre Empresa', 'Comuna del Proveedor', 'Región del Proveedor',
       'Observaciones', 'Forma de Pago', 'Orgcode_Comprador',
       'Entcode_Comprador', 'Orgcode_Proveedor', 'Entcode_Proveedor', 'year'],
      dtype='object')
Dimensi

,Nro Licitación Pública,Id Convenio Marco,Convenio Marco,CodigoOC,NombreOC,Fecha Envío OC,EstadoOC,Proviene de Gran Compra,idGranCompra,Especificación del Comprador,...,Nombre Empresa,Comuna del Proveedor,Región del Proveedor,Observaciones,Forma de Pago,Orgcode_Comprador,Entcode_Comprador,Orgcode_Proveedor,Entcode_Proveedor,year
0,2239-4-LR17,5800251.0,Compra de Vehículos Livianos y Medianos,2732-8-CM18,ADQUISICION DE MINIBUSES DEPARTAMENTO DE SALUD,2018-02-06,Aceptada,No,NaN,(1536899 )MINIBUS HYUNDAI H-1 MB 2.5 CRDI 6M/...,...,AUTOMOTORES GILDEMEISTER SPA,Vitacura,Región Metropolitana de Santiago,ADQUISICIÓN DE MINIBUSES PARA EL TRASLADO DE E...,30 días contra la recepción conforme de la fac...,3721.0,100137.0,32636,32413,2018
1,2239-4-LR17,5800251.0,Compra de Vehículos Livianos y Medianos,3232-132-CM18,HYUNDAI SANTA FÉ - USO MUNICIPAL SC 63893 - AD...,2018-02-08,Aceptada,No,NaN,(1536914 )SUV HYUNDAI SANTA FE DM WGN 2.4 6A/...,...,AUTOMOTORES GILDEMEISTER SPA,Vitacura,Región Metropolitana de Santiago,HYUNDAI SANTA FÉ - USO MUNICIPAL SC 63893 - AD...,30 días contra la recepción conforme de la fac...,4153.0,113981.0,32636,32413,2018


### Remove the observations that are not purchases of cars, but rather maintenance or leases. 

Do some cleaning based on Tipo de Producto

In [261]:
print(f"Shape of transac_df: {transac_df.shape}")
tipo_producto_values = transac_df['Tipo de Producto'].unique()
print(tipo_producto_values)

# Create a list of product types to drop
drop_product_types = ['ARRIENDO', 'MANTENCIÓN', 'OFICINAS', 'RODILLO',
                      'PESADO', 'BULL', 'TRASLADO', 'EXCAVA', 'TRACTO', 'ADORA', 'TOLVA', 'CARGADOR']

# Create a regex pattern to match any of the product types in the list
pattern = '|'.join(drop_product_types)

# Drop observations where 'Tipo de Producto' contains an element of the list
drop_mask = ~transac_df['Tipo de Producto'].str.contains(pattern, case=False, na=False)
transac_df = transac_df[drop_mask]
print(f"Shape of transac_df after dropping specified product types: {transac_df.shape}")



Shape of transac_df: (21434, 46)
['MINIBUS' 'SUV' 'MANTENCIÓN PREVENTIVA VEHÍCULO LIVIANO Y MEDIANO'
 'CAMIONETA' 'CARGO' 'SEDÁN' 'CARGADOR'
 'MANTENCIÓN PREVENTIVA VEHÍCULO PESADO' 'FURGÓN' 'RETRO EXCAVADORA'
 'HATCHBACK' 'MINIBUS PESADO' 'CAMIÓN LIVIANO' 'CITY CAR'
 'BUSES MEDIA Y LARGA DISTANCIA' 'TRACTOR' 'MOTO NIVELADORA'
 'CARRETILLA ELEVADORA' 'BULLDOZER' 'TOLVA' 'RODILLO COMPACTADOR'
 'EXCAVADORA' 'TAXIBÚS' 'ADITAMENTO PARA MAQUINARIA'
 'ARRIENDO LARGO PLAZO CAMIONETA' 'TRACTO' 'ARRIENDO LARGO PLAZO SEDÁN'
 'VEHÍCULOS POLICIALES' 'FURGÓN PESADO' 'AMBULANCIAS'
 'VEHÍCULO TRASLADO INTERNOS' 'ARRIENDO LARGO PLAZO FURGÓN'
 'ARRIENDO LARGO PLAZO CAMIÓN LIVIANO' 'ARRIENDO LARGO PLAZO SUV'
 'ARRIENDO LARGO PLAZO HATCHBACK' 'PAVIMENTADORA'
 'VEHÍCULO TRASLADO MENORES' 'ARRIENDO LARGO PLAZO ARRIENDO OFICINA MÓVIL'
 'ARRIENDO LARGO PLAZO MINIBUS' 'ARRIENDO LARGO PLAZO CITY CAR'
 'OFICINAS MOVILES' 'SEDí\x81N' 'FURGí\x93N' 'RETROEXCAVADORA'
 'BUS PESADO']
Shape of transac_df after droppin

In [262]:
tipo_producto_counts = transac_df['Tipo de Producto'].value_counts()
print(tipo_producto_counts)

Tipo de Producto
CAMIONETA                        2189
SUV                               908
MINIBUS                           380
CARGO                             265
SEDÁN                             234
AMBULANCIAS                       184
VEHÍCULOS POLICIALES              119
CAMIÓN LIVIANO                    103
FURGÓN                             87
BUSES MEDIA Y LARGA DISTANCIA      46
ADITAMENTO PARA MAQUINARIA         45
TAXIBÚS                            43
HATCHBACK                          41
CITY CAR                           29
SEDíN                             21
FURGíN                             8
Name: count, dtype: int64


In [263]:
# Normalize text by removing accents
transac_df['Tipo de Producto'] = transac_df['Tipo de Producto'].apply(
    lambda x: unicodedata.normalize('NFKD', x).encode('ASCII', 'ignore').decode('ASCII')
)

# Step 1: Create a mapping dictionary for all variations
replacements = {
    'SEDiN': 'SEDAN',
    'FURGiN' : 'FURGON'
}

# Step 2: Replace all variations using the mapping
transac_df['Tipo de Producto'] = transac_df['Tipo de Producto'].replace(replacements)

print( transac_df['Tipo de Producto'].value_counts())

Tipo de Producto
CAMIONETA                        2189
SUV                               908
MINIBUS                           380
CARGO                             265
SEDAN                             255
AMBULANCIAS                       184
VEHICULOS POLICIALES              119
CAMION LIVIANO                    103
FURGON                             95
BUSES MEDIA Y LARGA DISTANCIA      46
ADITAMENTO PARA MAQUINARIA         45
TAXIBUS                            43
HATCHBACK                          41
CITY CAR                           29
Name: count, dtype: int64


Do some cleaning based on 'Convenio Marco'

In [264]:
print(f"Shape of transac_df: {transac_df.shape}")
tipo_convenio = transac_df['Convenio Marco'].unique()
print(tipo_convenio)

avg_precio = transac_df.groupby('Convenio Marco')['Precio Unitario'].mean()
print(avg_precio)


# Remove products from FA of heavy and medium heavy vehicles in the Convenio Marco column
heavy_mask = ~transac_df['Convenio Marco'].str.contains('Compra de Vehículos Pesados y Maquinarias', na=False)
transac_df = transac_df[heavy_mask]
print(f"Shape of transac_df: {transac_df.shape}")


Shape of transac_df: (4702, 46)
['Compra de Vehículos Livianos y Medianos'
 'Compra de Vehículos Pesados y Maquinarias'
 'CM Adquisición de Vehículos SUV y Camionetas (Magento)'
 'CM Adquisición de Vehículos SUV y Camionetas'
 'Convenio Marco de  Adquisición de Vehículos']
Convenio Marco
CM Adquisición de Vehículos SUV y Camionetas              2.001232e+07
CM Adquisición de Vehículos SUV y Camionetas (Magento)    1.683101e+07
Compra de Vehículos Livianos y Medianos                   1.748541e+07
Compra de Vehículos Pesados y Maquinarias                 5.147216e+07
Convenio Marco de  Adquisición de Vehículos               2.091208e+07
Name: Precio Unitario, dtype: float64
Shape of transac_df: (4311, 46)


In [265]:
tipo_producto_counts = transac_df['Convenio Marco'].value_counts()
print(tipo_producto_counts)

Convenio Marco
Compra de Vehículos Livianos y Medianos                   2878
CM Adquisición de Vehículos SUV y Camionetas               951
Convenio Marco de  Adquisición de Vehículos                362
CM Adquisición de Vehículos SUV y Camionetas (Magento)     120
Name: count, dtype: int64


### Pair the variables of the model with the variables of the data     
        
    'IDProductoCM' -> j 
    To create product characteristics: 'Tipo de Producto', 'Marca', 'Nombre Producto ONU'
    'Precio Unitario'-> net not including taxes, just changes the scale of the price parameter 
    'Rut Unidad de Compra' -> i 
    To create the groups (k):  'Región Unidad de Compra', 'Sector'
     'year' -> t 

     'Modelo'-> to do the match with the product characteristics. 
        
Variables not directly paired but which could be useful: 
    'Nro Licitación Pública', 'Id Convenio Marco', 'Convenio Marco', 'CodigoOC', 'Fecha Envío OC', 'Cantidad', 'Rut Proveedor', 
    'Nombre Proveedor Sucursal', 'Orgcode_Comprador', 'Entcode_Comprador', 


In [266]:
# Variables to keep - directly paired with model
model_vars = ['IDProductoCM', 'Modelo',  'Tipo de Producto', 'Marca', 'Nombre Producto ONU',
              'Precio Unitario', 'Rut Unidad de Compra', 'Región Unidad de Compra',
              'Sector', 'year']

# Additional useful variables
extra_vars = ['Nro Licitación Pública', 'Convenio Marco', 'CodigoOC',
              'Fecha Envío OC', 'Cantidad', 'Rut Proveedor',
              'Nombre Proveedor Sucursal']

# Create new dataframe with only the specified columns
transac_df = transac_df[ model_vars + extra_vars]



print(f"Dimensions of transac_df: {transac_df.shape}")
transac_df.head(2)
show(transac_df)


PandasGUI INFO — pandasgui.gui — Opening PandasGUI


Dimensions of transac_df: (4311, 17)


In [267]:
dest_path = os.path.abspath(os.path.join('..', 'interm_data', 'yearly_data', 'Transacciones', 'joined_cleaned_transac.csv'))
print(f"Absolute path of dest_path: {dest_path}")
transac_df.to_csv(dest_path, index=False)

Absolute path of dest_path: c:\Users\lucas\OneDrive - Yale University\Documents\GitHub\2nd-year-paper\interm_data\yearly_data\Transacciones\joined_cleaned_transac.csv


store the data

Answer the following questions 
- are firms selling multiple products? Yes


In [268]:
# Group by supplier and count unique products
supplier_products = transac_df.groupby('Nombre Proveedor Sucursal')['IDProductoCM'].nunique().sort_values(ascending=False)
print(supplier_products)

n_unique_products = transac_df['IDProductoCM'].nunique()
print(f"Total number of unique products: {n_unique_products}")

Nombre Proveedor Sucursal
Salinas y Fabres S.A.                            296
SALAZAR ISRAEL                                   204
Automotores Gildemeister CAMIONES                187
Dercosa                                          153
BRUNO FRITSCH S.A.                               115
                                                ... 
SUBARU CHILE S.A.                                  1
AUTOMOTORES FRANCO CHILENA  S.A                    1
E. Kovacs SpA                                      1
Salinas y Fabres S.A. - Sucursal Punta Arenas      1
Frontera                                           1
Name: IDProductoCM, Length: 65, dtype: int64
Total number of unique products: 1419


### Merge product characteristics (still to do, Pablo has to send me the code)

In [107]:
# Assuming transac_df is already defined and loaded with data
unique_modelo_values = transac_df['Modelo'].unique()
print(unique_modelo_values)

['H-1  MB 2.5 CRDI 6M/T GLS 10S AC 2AB ABS'
 'SANTA FE DM  WGN 2.4 6A/T 4WD GLS FULL PE'
 'SANTA FE DM  WGN 2.4 6A/T GLS PE' ...
 'PALISADE LX2 3.5 AWD PREMIUM FL AT 2024' 'COLORADO LT 4X4 AT 2024'
 'D60 ELITE 7DCT 4X2 AT 2024']


In [67]:
# import the product characteristics
prod_char_path = os.path.join('..', 'car_data',   'scraped_data.csv')
prod_char_df = pd.read_csv(prod_char_path)
prod_char_df.rename(columns={'MODEL': 'Modelo'}, inplace=True)

prod_char_df['matching var'] = prod_char_df['MARCA'] + ' ' + prod_char_df['Model Name'] + ' ' + prod_char_df['Drive']


prod_char_df.head()





,Modelo,MARCA,Model Name,Engine,Drive,Transmission,Year,Version URL,Motor - Combustible,Motor - Cilindrada,...,Motor eléctrico - Voltaje,Confort - Palanca de cambios,Transmisión y chasis - Neumáticos delanteros,Transmisión y chasis - Neumáticos traseros,Motor - Potencia total sistema híbrido,Seguridad - Cabeceras delanteras activas,Comunicación y entretenimiento - Preinstalación teléfono,Comunicación y entretenimiento - Disco rígido,Confort - Puertas,matching var
0,CHANGAN HUNTER COMFORT 2.0T 4X2 MT 2024,CHANGAN,HUNTER COMFORT,2.0T,4X2,MT,2024,https://www.autocosmos.cl/catalogo/2024/changa...,diesel,1910 cc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHANGAN HUNTER COMFORT 4X2
1,CHANGAN HUNTER COMFORT 2.0T 4X4 MT 2024,CHANGAN,HUNTER COMFORT,2.0T,4X4,MT,2024,https://www.autocosmos.cl/catalogo/2024/changa...,diesel,1910 cc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHANGAN HUNTER COMFORT 4X4
2,CHANGAN HUNTER ELITE SPORT 2.0T 4X4 MT 2024,CHANGAN,HUNTER ELITE SPORT,2.0T,4X4,MT,2024,https://www.autocosmos.cl/catalogo/2024/changa...,diesel,1910 cc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHANGAN HUNTER ELITE SPORT 4X4
3,CHANGAN HUNTER LUXURY 2.0T 4X2 MT 2024,CHANGAN,HUNTER LUXURY,2.0T,4X2,MT,2024,https://www.autocosmos.cl/catalogo/2024/changa...,diesel,1910 cc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHANGAN HUNTER LUXURY 4X2
4,CHANGAN MD201 MT 2024,CHANGAN,MD201,NaN,NaN,MT,2024,https://www.autocosmos.cl/catalogo/2024/changa...,bencina,1243 cc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
transac_df['matching var'] = transac_df['Marca'] + ' ' + transac_df['Modelo']

# Get number of rows before merge
n_before = len(transac_df)

# Merge dataframes
transac_df = transac_df.merge(prod_char_df, on='matching var', how='left')

# Get number of rows after merge
n_after = len(transac_df)

# Get number of matched rows by checking any column from prod_char_df
# Replace 'some_column_from_prod_char_df' with an actual column name from prod_char_df
n_matched = transac_df['Version URL'].notna().sum()

print(f"Observations before merge: {n_before}")
print(f"Observations after merge: {n_after}")
print(f"Number of matched observations: {n_matched}")
print(f"Number of unmatched observations: {n_after - n_matched}")
print(f"Percentage of matched observations: {(n_matched/n_after*100):.2f}%")

KeyError: 'Modelo'

# Demand estimation 

In [124]:
# create the k groups based on 'Region Unidad de Compra' and 'Sector' 
transac_df['k_group'] = transac_df.groupby(['Sector', 'Región Unidad de Compra']).ngroup()
#

In [125]:
transac_df

,IDProductoCM,Modelo,Tipo de Producto,Marca,Nombre Producto ONU,Precio Unitario,Rut Unidad de Compra,Región Unidad de Compra,Sector,year,Nro Licitación Pública,Id Convenio Marco,Convenio Marco,CodigoOC,Fecha Envío OC,Cantidad,Rut Proveedor,Nombre Proveedor Sucursal,k_group
0,1536899,H-1 MB 2.5 CRDI 6M/T GLS 10S AC 2AB ABS,MINIBUS,HYUNDAI,Minibuses,17098600.0,69.071.700-K,Metropolitana,Municipalidades,2018,2239-4-LR17,5800251.0,Compra de Vehículos Livianos y Medianos,2732-8-CM18,2018-02-06,2.0,79.649.140-k,Automotores Gildemeister CAMIONES,48
1,1536914,SANTA FE DM WGN 2.4 6A/T 4WD GLS FULL PE,SUV,HYUNDAI,Automóviles,19740000.0,69.073.500-8,Valparaíso,Municipalidades,2018,2239-4-LR17,5800251.0,Compra de Vehículos Livianos y Medianos,3232-132-CM18,2018-02-08,1.0,79.649.140-k,Automotores Gildemeister CAMIONES,50
2,1536916,SANTA FE DM WGN 2.4 6A/T GLS PE,SUV,HYUNDAI,Automóviles,14921513.0,65.154.016-k,Metropolitana,"Gob. Central, Universidades",2018,2239-4-LR17,5800251.0,Compra de Vehículos Livianos y Medianos,1002584-16-CM18,2018-02-08,1.0,79.649.140-k,Automotores Gildemeister CAMIONES,24
3,1536916,SANTA FE DM WGN 2.4 6A/T GLS PE,SUV,HYUNDAI,Automóviles,14921513.0,65.154.021-6,Metropolitana,"Gob. Central, Universidades",2018,2239-4-LR17,5800251.0,Compra de Vehículos Livianos y Medianos,1002592-43-CM18,2018-02-08,1.0,79.649.140-k,Automotores Gildemeister CAMIONES,24
4,1536979,NEW MAHINDRA SCORPIO SUV 4X4 ABS 2AB,SUV,MAHINDRA,Automóviles,10497983.0,61.955.100-1,Araucanía,Salud,2018,2239-4-LR17,5800251.0,Compra de Vehículos Livianos y Medianos,4725-21-CM18,2018-02-09,1.0,79.649.140-k,Automotores Gildemeister CAMIONES,86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21429,2049176,V700 RAPID 1.4L 2024,FURGíN,RAM,Mini-furgonetas o furgonetas,12410800.0,65.154.398-3,Tarapacá,"Gob. Central, Universidades",2024,2239-8-LR23,5802342.0,Convenio Marco de Adquisición de Vehículos,1026727-301-CM24,2024-08-30,1.0,91.502.000-3,Salinas y Fabres S.A.,25
21430,2054456,D60 ELITE 7DCT 4X2 AT 2024,SUV,MAXUS,Automóviles,16403950.0,69.130.600-3,Maule,Municipalidades,2024,2239-8-LR23,5802342.0,Convenio Marco de Adquisición de Vehículos,4085-605-CM24,2024-08-30,3.0,96.889.440-4,Mediterraneo Automotores SA,47
21431,2048705,T8 CE COMFORT 2.0T 4X2 MT 2024,CAMIONETA,JAC,Automóviles,15541506.0,69.071.800-6,Metropolitana,Municipalidades,2024,2239-8-LR23,5802342.0,Convenio Marco de Adquisición de Vehículos,2702-467-CM24,2024-08-30,1.0,86.385.500-4,COMERCIAL ANFRUNS Y COMPAíIA SPA,48
21432,2048790,T8 PRO LUXURY 4X4 MT 2024,CAMIONETA,JAC,Automóviles,18481640.0,69.071.800-6,Metropolitana,Municipalidades,2024,2239-8-LR23,5802342.0,Convenio Marco de Adquisición de Vehículos,2702-468-CM24,2024-08-30,1.0,86.385.500-4,COMERCIAL ANFRUNS Y COMPAíIA SPA,48


In [127]:
# Count the occurrences of each 'Rut Unidad de Compra'
rut_counts = transac_df['Rut Unidad de Compra'].value_counts()

# Create a new column in transac_df to store the counts
transac_df['Rut Count'] = transac_df['Rut Unidad de Compra'].map(rut_counts)

# Sort the dataframe by the 'Rut Count' column in descending order
transac_df = transac_df.sort_values(by='Rut Count', ascending=False)

# Drop the 'Rut Count' column if it's no longer needed
transac_df = transac_df.drop(columns=['Rut Count'])

# Reset the index if needed
transac_df = transac_df.reset_index(drop=True)

show(transac_df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
